# Cyber Crime Science Data Analysis and Visualization Project

## Data Import

### Libraries

In [1]:
import pandas as pd
import numpy as np


In [2]:
slachtofferschap_source = "data/source/slachtofferschap.csv"
age_distribution_source = "data/source/age_distribution.csv"
digitale_vaardigheden_source = "data/source/digitale_vaardigheden.csv"
education_levels_source = "data/source/education_levels.csv"
gdp_source = "data/source/gdp.csv"
bevolkingdichtheid_source = "data/source/bevolkingdichtheid.csv"
vestigingen_source = "data/source/vestigingen.csv"

In [3]:
slachtofferschap = pd.read_csv(slachtofferschap_source, sep=';')
age_distribution = pd.read_csv(age_distribution_source, sep=';')
digitale_vaardigheden = pd.read_csv(digitale_vaardigheden_source, sep=',')
education_levels = pd.read_csv(education_levels_source, sep=';')
gdp = pd.read_csv(gdp_source, sep=';')
bevolkingdichtheid = pd.read_csv(bevolkingdichtheid_source, sep=';')
vestigingen = pd.read_csv(vestigingen_source, sep=';')

## Data Cleaning

### Keep only the desired columns

In [4]:
# Calculate HighRiskShare for each province
vestigingen["HighRiskShare"] = (
    (vestigingen["J Informatie en communicatie"] + 
     vestigingen["K Financiële dienstverlening"] + 
     vestigingen["Q Gezondheids- en welzijnszorg"]) / 
    vestigingen["Totaal"]
) * 100

# Keep oply the "Regio's", "HighRiskShare" columns
vestigingen = vestigingen[["Regio's", "HighRiskShare"]]

In [5]:
bevolkingdichtheid_filtered = bevolkingdichtheid[[
    "Regio's",
    "Bevolking/Bevolkingssamenstelling op 1 januari/Bevolkingsdichtheid (aantal inwoners per km²)"
]]

bevolkingdichtheid_filtered = bevolkingdichtheid_filtered.rename(columns={
    "Bevolking/Bevolkingssamenstelling op 1 januari/Bevolkingsdichtheid (aantal inwoners per km²)": "Bevolkingsdichtheid (aantal inwoners per km²)"
})

In [6]:
gdp_filtered = gdp[[
    "Regio's",
    "Bbp (marktprijzen) (mln euro)",
    "Bbp per inwoner (euro)",
    "Bbp, volumemutaties (%)",
    "Bruto toegevoegde waarde, volumemutaties (%)",
    "Toegevoegde waarde (bruto, basisprijzen) (mln euro)",
]]

In [7]:
education_levels_filtered = education_levels[[
    "Regio's",
    "Onderwijsniveau 5 categorieën/11 Basisonderwijs (%)",
    "Onderwijsniveau 5 categorieën/12 Vmbo, havo-, vwo-onderbouw, mbo1 (%)",
    "Onderwijsniveau 5 categorieën/21 Havo, vwo, mbo2-4 (%)",
    "Onderwijsniveau 5 categorieën/31 Hbo-, wo-bachelor (%)",
    "Onderwijsniveau 5 categorieën/32 Hbo-, wo-master, doctor (%)",
]]

education_levels_filtered = education_levels_filtered.rename(columns={
    "Regio's": "Regio's",
    "Onderwijsniveau 5 categorieën/11 Basisonderwijs (%)": "Basisonderwijs (%)",
    "Onderwijsniveau 5 categorieën/12 Vmbo, havo-, vwo-onderbouw, mbo1 (%)": "Vmbo/Havo/Vwo/Mbo1 (%)",
    "Onderwijsniveau 5 categorieën/21 Havo, vwo, mbo2-4 (%)": "Havo/Vwo/Mbo2-4 (%)",
    "Onderwijsniveau 5 categorieën/31 Hbo-, wo-bachelor (%)": "Hbo/Wo-bachelor (%)",
    "Onderwijsniveau 5 categorieën/32 Hbo-, wo-master, doctor (%)": "Hbo/Wo-master/Doctor (%)"
})

In [8]:
age_distribution = age_distribution.applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)
age_distribution.columns = age_distribution.columns.str.replace('"', '')

# Strip whitespace from column names to ensure proper matching
age_distribution.columns = age_distribution.columns.str.strip()

# Drop the 'Leeftijd' column
age_distribution_filtered = age_distribution.drop(columns=['Leeftijd'])

/var/folders/b_/0xr02dzn7x98hhm9_vd5w_900000gn/T/ipykernel_29443/690944268.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  age_distribution = age_distribution.applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)


In [9]:
# Strip whitespace from column names to ensure proper matching
slachtofferschap.columns = slachtofferschap.columns.str.strip()

# Drop the 'Leeftijd' column
slachtofferschap_filtered = slachtofferschap.drop(columns=['Marges', 'Perioden'])

In [10]:
digitale_vaardigheden = digitale_vaardigheden.applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)
digitale_vaardigheden.columns = digitale_vaardigheden.columns.str.replace('"', '')

display(digitale_vaardigheden)

/var/folders/b_/0xr02dzn7x98hhm9_vd5w_900000gn/T/ipykernel_29443/2641234680.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  digitale_vaardigheden = digitale_vaardigheden.applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)


,Regio's,Meer dan basisvaardigheden,Basisvaardigheden,Geringe/geen digitale vaardigheden,Kan niet vastgeteld worden
0,Groningen (PV),"43,8","35,1","17,8","3,3"
1,Fryslân (PV),"44,3","33,9","14,8","7,0"
2,Drenthe (PV),"44,9","29,1","21,6","4,4"
3,Overijssel (PV),"47,9","28,4","20,9","2,8"
4,Flevoland (PV),"55,0","26,1","17,4",.
5,Gelderland (PV),"46,4","30,7","19,8","3,0"
6,Utrecht (PV),"57,6","26,7","13,7","2,0"
7,Noord-Holland (PV),"53,6","28,7","15,9","1,8"
8,Zuid-Holland (PV),"51,5","27,2","18,3","2,9"
9,Zeeland (PV),"39,9","28,3","28,7","3,1"


## Merge DataFrames

In [11]:
merged_df = pd.merge(
    slachtofferschap_filtered,
    age_distribution_filtered,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

display(merged_df)

,Regio's,Online criminaliteit/Totaal slachtoffers (%),Online oplichting en fraude/Totaal slachtoffers (%),Online oplichting en fraude/Aankoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Verkoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Fraude betalingsverkeer/Totaal slachtoffers (%),Online oplichting en fraude/Identiteitsfraude/Totaal slachtoffers (%),Online oplichting en fraude/Phishing/Totaal slachtoffers (%),Hacken/Totaal slachtoffers (%),Hacken/Apparaat/Totaal slachtoffers (%),...,81 jaar,82 jaar,83 jaar,84 jaar,85 jaar,86 jaar,87 jaar,88 jaar,89 jaar,95 jaar of ouder
0,Groningen (PV),16.3,8.6,6.5,1.4,1.0,0.6,1.0,6.3,2.7,...,"3655,0","3173,0","2921,0","2747,5","2482,0","2161,0","1979,0","1709,0","1485,5","1099,0"
1,Fryslân (PV),14.2,8.5,6.7,1.3,0.8,0.6,0.6,5.0,2.3,...,"4637,0","4113,5","3831,5","3508,5","3199,0","2826,0","2533,5","2115,5","1775,0","1347,0"
2,Drenthe (PV),14.8,8.8,6.6,1.7,1.4,0.6,1.0,5.9,2.8,...,"3756,0","3374,0","3150,0","2834,5","2619,5","2197,5","2022,5","1742,5","1531,0","1089,5"
3,Overijssel (PV),14.8,9.0,6.8,1.3,0.8,0.6,0.6,5.4,2.3,...,"7311,5","6434,0","6293,5","5740,0","5171,0","4535,5","3909,0","3318,5","2845,5","2198,5"
4,Flevoland (PV),16.9,10.1,7.6,1.7,1.0,0.8,0.9,5.5,2.8,...,"1673,5","1424,0","1294,0","1215,0","1080,0","870,0","820,5","711,5","600,0","435,5"
5,Gelderland (PV),14.7,8.4,6.3,1.2,0.8,0.6,0.7,5.3,2.2,...,"13367,0","12218,0","11700,5","10417,5","9609,5","7937,5","7178,5","6276,0","5432,0","3918,5"
6,Utrecht (PV),15.0,9.0,6.7,1.4,1.2,0.8,0.7,5.4,2.1,...,"7143,0","6544,5","6420,5","5724,5","5200,5","4517,0","3927,5","3434,0","2936,0","2336,0"
7,Noord-Holland (PV),16.9,10.0,7.5,1.5,1.3,0.7,0.9,5.8,2.4,...,"16268,0","14708,5","14414,0","12985,0","11616,5","9677,5","8654,0","7498,0","6254,5","5105,5"
8,Zuid-Holland (PV),15.8,9.5,6.9,1.4,1.1,0.8,0.9,5.5,2.3,...,"20362,0","18946,0","18784,0","16823,5","15262,0","12832,0","11379,5","10065,0","8513,5","7113,0"
9,Zeeland (PV),15.8,9.8,7.4,1.8,0.9,0.7,1.0,5.5,2.4,...,"2938,0","2607,5","2459,5","2356,5","2085,5","1805,5","1609,5","1418,5","1238,0","1125,5"


In [12]:
merged_df = pd.merge(
    merged_df,
    education_levels_filtered,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

display(merged_df)

,Regio's,Online criminaliteit/Totaal slachtoffers (%),Online oplichting en fraude/Totaal slachtoffers (%),Online oplichting en fraude/Aankoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Verkoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Fraude betalingsverkeer/Totaal slachtoffers (%),Online oplichting en fraude/Identiteitsfraude/Totaal slachtoffers (%),Online oplichting en fraude/Phishing/Totaal slachtoffers (%),Hacken/Totaal slachtoffers (%),Hacken/Apparaat/Totaal slachtoffers (%),...,86 jaar,87 jaar,88 jaar,89 jaar,95 jaar of ouder,Basisonderwijs (%),Vmbo/Havo/Vwo/Mbo1 (%),Havo/Vwo/Mbo2-4 (%),Hbo/Wo-bachelor (%),Hbo/Wo-master/Doctor (%)
0,Groningen (PV),16.3,8.6,6.5,1.4,1.0,0.6,1.0,6.3,2.7,...,"2161,0","1979,0","1709,0","1485,5","1099,0",7.4,16.3,45.1,20.0,11.2
1,Fryslân (PV),14.2,8.5,6.7,1.3,0.8,0.6,0.6,5.0,2.3,...,"2826,0","2533,5","2115,5","1775,0","1347,0",7.8,19.1,47.0,19.7,6.4
2,Drenthe (PV),14.8,8.8,6.6,1.7,1.4,0.6,1.0,5.9,2.8,...,"2197,5","2022,5","1742,5","1531,0","1089,5",8.4,18.9,46.5,18.7,7.6
3,Overijssel (PV),14.8,9.0,6.8,1.3,0.8,0.6,0.6,5.4,2.3,...,"4535,5","3909,0","3318,5","2845,5","2198,5",8.2,18.1,44.9,20.3,8.5
4,Flevoland (PV),16.9,10.1,7.6,1.7,1.0,0.8,0.9,5.5,2.8,...,"870,0","820,5","711,5","600,0","435,5",8.6,18.5,45.6,19.6,7.6
5,Gelderland (PV),14.7,8.4,6.3,1.2,0.8,0.6,0.7,5.3,2.2,...,"7937,5","7178,5","6276,0","5432,0","3918,5",8.4,18.1,42.3,20.1,11.1
6,Utrecht (PV),15.0,9.0,6.7,1.4,1.2,0.8,0.7,5.4,2.1,...,"4517,0","3927,5","3434,0","2936,0","2336,0",7.6,15.0,36.1,22.2,19.0
7,Noord-Holland (PV),16.9,10.0,7.5,1.5,1.3,0.7,0.9,5.8,2.4,...,"9677,5","8654,0","7498,0","6254,5","5105,5",8.3,15.7,37.7,21.6,16.7
8,Zuid-Holland (PV),15.8,9.5,6.9,1.4,1.1,0.8,0.9,5.5,2.3,...,"12832,0","11379,5","10065,0","8513,5","7113,0",10.0,17.5,40.0,19.4,13.2
9,Zeeland (PV),15.8,9.8,7.4,1.8,0.9,0.7,1.0,5.5,2.4,...,"1805,5","1609,5","1418,5","1238,0","1125,5",8.8,20.0,47.1,17.6,6.6


In [13]:
merged_df = pd.merge(
    merged_df,
    bevolkingdichtheid_filtered,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

display(merged_df)

,Regio's,Online criminaliteit/Totaal slachtoffers (%),Online oplichting en fraude/Totaal slachtoffers (%),Online oplichting en fraude/Aankoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Verkoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Fraude betalingsverkeer/Totaal slachtoffers (%),Online oplichting en fraude/Identiteitsfraude/Totaal slachtoffers (%),Online oplichting en fraude/Phishing/Totaal slachtoffers (%),Hacken/Totaal slachtoffers (%),Hacken/Apparaat/Totaal slachtoffers (%),...,87 jaar,88 jaar,89 jaar,95 jaar of ouder,Basisonderwijs (%),Vmbo/Havo/Vwo/Mbo1 (%),Havo/Vwo/Mbo2-4 (%),Hbo/Wo-bachelor (%),Hbo/Wo-master/Doctor (%),Bevolkingsdichtheid (aantal inwoners per km²)
0,Groningen (PV),16.3,8.6,6.5,1.4,1.0,0.6,1.0,6.3,2.7,...,"1979,0","1709,0","1485,5","1099,0",7.4,16.3,45.1,20.0,11.2,257
1,Fryslân (PV),14.2,8.5,6.7,1.3,0.8,0.6,0.6,5.0,2.3,...,"2533,5","2115,5","1775,0","1347,0",7.8,19.1,47.0,19.7,6.4,197
2,Drenthe (PV),14.8,8.8,6.6,1.7,1.4,0.6,1.0,5.9,2.8,...,"2022,5","1742,5","1531,0","1089,5",8.4,18.9,46.5,18.7,7.6,191
3,Overijssel (PV),14.8,9.0,6.8,1.3,0.8,0.6,0.6,5.4,2.3,...,"3909,0","3318,5","2845,5","2198,5",8.2,18.1,44.9,20.3,8.5,357
4,Flevoland (PV),16.9,10.1,7.6,1.7,1.0,0.8,0.9,5.5,2.8,...,"820,5","711,5","600,0","435,5",8.6,18.5,45.6,19.6,7.6,315
5,Gelderland (PV),14.7,8.4,6.3,1.2,0.8,0.6,0.7,5.3,2.2,...,"7178,5","6276,0","5432,0","3918,5",8.4,18.1,42.3,20.1,11.1,430
6,Utrecht (PV),15.0,9.0,6.7,1.4,1.2,0.8,0.7,5.4,2.1,...,"3927,5","3434,0","2936,0","2336,0",7.6,15.0,36.1,22.2,19.0,935
7,Noord-Holland (PV),16.9,10.0,7.5,1.5,1.3,0.7,0.9,5.8,2.4,...,"8654,0","7498,0","6254,5","5105,5",8.3,15.7,37.7,21.6,16.7,1109
8,Zuid-Holland (PV),15.8,9.5,6.9,1.4,1.1,0.8,0.9,5.5,2.3,...,"11379,5","10065,0","8513,5","7113,0",10.0,17.5,40.0,19.4,13.2,1410
9,Zeeland (PV),15.8,9.8,7.4,1.8,0.9,0.7,1.0,5.5,2.4,...,"1609,5","1418,5","1238,0","1125,5",8.8,20.0,47.1,17.6,6.6,220


In [14]:
vestigingen["Regio's"] = vestigingen["Regio's"].str.strip()
merged_df["Regio's"] = merged_df["Regio's"].str.strip()

merged_df = pd.merge(
    merged_df,
    vestigingen,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

display(merged_df)

,Regio's,Online criminaliteit/Totaal slachtoffers (%),Online oplichting en fraude/Totaal slachtoffers (%),Online oplichting en fraude/Aankoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Verkoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Fraude betalingsverkeer/Totaal slachtoffers (%),Online oplichting en fraude/Identiteitsfraude/Totaal slachtoffers (%),Online oplichting en fraude/Phishing/Totaal slachtoffers (%),Hacken/Totaal slachtoffers (%),Hacken/Apparaat/Totaal slachtoffers (%),...,88 jaar,89 jaar,95 jaar of ouder,Basisonderwijs (%),Vmbo/Havo/Vwo/Mbo1 (%),Havo/Vwo/Mbo2-4 (%),Hbo/Wo-bachelor (%),Hbo/Wo-master/Doctor (%),Bevolkingsdichtheid (aantal inwoners per km²),HighRiskShare
0,Groningen (PV),16.3,8.6,6.5,1.4,1.0,0.6,1.0,6.3,2.7,...,"1709,0","1485,5","1099,0",7.4,16.3,45.1,20.0,11.2,257,18.563724
1,Fryslân (PV),14.2,8.5,6.7,1.3,0.8,0.6,0.6,5.0,2.3,...,"2115,5","1775,0","1347,0",7.8,19.1,47.0,19.7,6.4,197,16.081363
2,Drenthe (PV),14.8,8.8,6.6,1.7,1.4,0.6,1.0,5.9,2.8,...,"1742,5","1531,0","1089,5",8.4,18.9,46.5,18.7,7.6,191,18.672477
3,Overijssel (PV),14.8,9.0,6.8,1.3,0.8,0.6,0.6,5.4,2.3,...,"3318,5","2845,5","2198,5",8.2,18.1,44.9,20.3,8.5,357,19.470765
4,Flevoland (PV),16.9,10.1,7.6,1.7,1.0,0.8,0.9,5.5,2.8,...,"711,5","600,0","435,5",8.6,18.5,45.6,19.6,7.6,315,20.733450
5,Gelderland (PV),14.7,8.4,6.3,1.2,0.8,0.6,0.7,5.3,2.2,...,"6276,0","5432,0","3918,5",8.4,18.1,42.3,20.1,11.1,430,20.371861
6,Utrecht (PV),15.0,9.0,6.7,1.4,1.2,0.8,0.7,5.4,2.1,...,"3434,0","2936,0","2336,0",7.6,15.0,36.1,22.2,19.0,935,22.358339
7,Noord-Holland (PV),16.9,10.0,7.5,1.5,1.3,0.7,0.9,5.8,2.4,...,"7498,0","6254,5","5105,5",8.3,15.7,37.7,21.6,16.7,1109,19.573005
8,Zuid-Holland (PV),15.8,9.5,6.9,1.4,1.1,0.8,0.9,5.5,2.3,...,"10065,0","8513,5","7113,0",10.0,17.5,40.0,19.4,13.2,1410,19.231222
9,Zeeland (PV),15.8,9.8,7.4,1.8,0.9,0.7,1.0,5.5,2.4,...,"1418,5","1238,0","1125,5",8.8,20.0,47.1,17.6,6.6,220,17.124105


In [15]:
merged_df = pd.merge(
    merged_df,
    gdp_filtered,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

display(merged_df)

,Regio's,Online criminaliteit/Totaal slachtoffers (%),Online oplichting en fraude/Totaal slachtoffers (%),Online oplichting en fraude/Aankoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Verkoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Fraude betalingsverkeer/Totaal slachtoffers (%),Online oplichting en fraude/Identiteitsfraude/Totaal slachtoffers (%),Online oplichting en fraude/Phishing/Totaal slachtoffers (%),Hacken/Totaal slachtoffers (%),Hacken/Apparaat/Totaal slachtoffers (%),...,Havo/Vwo/Mbo2-4 (%),Hbo/Wo-bachelor (%),Hbo/Wo-master/Doctor (%),Bevolkingsdichtheid (aantal inwoners per km²),HighRiskShare,Bbp (marktprijzen) (mln euro),Bbp per inwoner (euro),"Bbp, volumemutaties (%)","Bruto toegevoegde waarde, volumemutaties (%)","Toegevoegde waarde (bruto, basisprijzen) (mln euro)"
0,Groningen (PV),16.3,8.6,6.5,1.4,1.0,0.6,1.0,6.3,2.7,...,45.1,20.0,11.2,257,18.563724,32566,55332,"1,7","2,1",29211
1,Fryslân (PV),14.2,8.5,6.7,1.3,0.8,0.6,0.6,5.0,2.3,...,47.0,19.7,6.4,197,16.081363,25490,39052,"3,5",4,22864
2,Drenthe (PV),14.8,8.8,6.6,1.7,1.4,0.6,1.0,5.9,2.8,...,46.5,18.7,7.6,191,18.672477,18434,37146,"2,8","3,2",16535
3,Overijssel (PV),14.8,9.0,6.8,1.3,0.8,0.6,0.6,5.4,2.3,...,44.9,20.3,8.5,357,19.470765,53172,45476,"3,8","4,2",47694
4,Flevoland (PV),16.9,10.1,7.6,1.7,1.0,0.8,0.9,5.5,2.8,...,45.6,19.6,7.6,315,20.733450,17303,40100,"4,3","4,8",15521
5,Gelderland (PV),14.7,8.4,6.3,1.2,0.8,0.6,0.7,5.3,2.2,...,42.3,20.1,11.1,430,20.371861,95416,45360,"3,8","4,3",85586
6,Utrecht (PV),15.0,9.0,6.7,1.4,1.2,0.8,0.7,5.4,2.1,...,36.1,22.2,19.0,935,22.358339,88660,64928,4,"4,5",79527
7,Noord-Holland (PV),16.9,10.0,7.5,1.5,1.3,0.7,0.9,5.8,2.4,...,37.7,21.6,16.7,1109,19.573005,203766,70285,"6,7","7,2",182774
8,Zuid-Holland (PV),15.8,9.5,6.9,1.4,1.1,0.8,0.9,5.5,2.3,...,40.0,19.4,13.2,1410,19.231222,200451,53597,"3,5",4,179802
9,Zeeland (PV),15.8,9.8,7.4,1.8,0.9,0.7,1.0,5.5,2.4,...,47.1,17.6,6.6,220,17.124105,17839,46205,"4,7","5,2",16001


In [16]:
merged_df = pd.merge(
    merged_df,
    digitale_vaardigheden,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

display(merged_df)

,Regio's,Online criminaliteit/Totaal slachtoffers (%),Online oplichting en fraude/Totaal slachtoffers (%),Online oplichting en fraude/Aankoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Verkoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Fraude betalingsverkeer/Totaal slachtoffers (%),Online oplichting en fraude/Identiteitsfraude/Totaal slachtoffers (%),Online oplichting en fraude/Phishing/Totaal slachtoffers (%),Hacken/Totaal slachtoffers (%),Hacken/Apparaat/Totaal slachtoffers (%),...,HighRiskShare,Bbp (marktprijzen) (mln euro),Bbp per inwoner (euro),"Bbp, volumemutaties (%)","Bruto toegevoegde waarde, volumemutaties (%)","Toegevoegde waarde (bruto, basisprijzen) (mln euro)",Meer dan basisvaardigheden,Basisvaardigheden,Geringe/geen digitale vaardigheden,Kan niet vastgeteld worden
0,Groningen (PV),16.3,8.6,6.5,1.4,1.0,0.6,1.0,6.3,2.7,...,18.563724,32566,55332,"1,7","2,1",29211,"43,8","35,1","17,8","3,3"
1,Fryslân (PV),14.2,8.5,6.7,1.3,0.8,0.6,0.6,5.0,2.3,...,16.081363,25490,39052,"3,5",4,22864,"44,3","33,9","14,8","7,0"
2,Drenthe (PV),14.8,8.8,6.6,1.7,1.4,0.6,1.0,5.9,2.8,...,18.672477,18434,37146,"2,8","3,2",16535,"44,9","29,1","21,6","4,4"
3,Overijssel (PV),14.8,9.0,6.8,1.3,0.8,0.6,0.6,5.4,2.3,...,19.470765,53172,45476,"3,8","4,2",47694,"47,9","28,4","20,9","2,8"
4,Flevoland (PV),16.9,10.1,7.6,1.7,1.0,0.8,0.9,5.5,2.8,...,20.733450,17303,40100,"4,3","4,8",15521,"55,0","26,1","17,4",.
5,Gelderland (PV),14.7,8.4,6.3,1.2,0.8,0.6,0.7,5.3,2.2,...,20.371861,95416,45360,"3,8","4,3",85586,"46,4","30,7","19,8","3,0"
6,Utrecht (PV),15.0,9.0,6.7,1.4,1.2,0.8,0.7,5.4,2.1,...,22.358339,88660,64928,4,"4,5",79527,"57,6","26,7","13,7","2,0"
7,Noord-Holland (PV),16.9,10.0,7.5,1.5,1.3,0.7,0.9,5.8,2.4,...,19.573005,203766,70285,"6,7","7,2",182774,"53,6","28,7","15,9","1,8"
8,Zuid-Holland (PV),15.8,9.5,6.9,1.4,1.1,0.8,0.9,5.5,2.3,...,19.231222,200451,53597,"3,5",4,179802,"51,5","27,2","18,3","2,9"
9,Zeeland (PV),15.8,9.8,7.4,1.8,0.9,0.7,1.0,5.5,2.4,...,17.124105,17839,46205,"4,7","5,2",16001,"39,9","28,3","28,7","3,1"


In [ ]:
#TO DO:

#gdp merge : Done
#age distribution : Done
#formule toevoegen over die vestigingen: Done
#Digital Skills toevoegen: Done
#Voeg slachtofferschap scores voor andere crimes ook toe: Done



In [ ]:
#merged_df.to_csv("data/save/merged_data.csv", index=False)

## Analyse (To Do)

In [ ]:
##Analyse
##Heatmaps: correlatie tussen verschillende variabelen met map van Nederland
##
#
## Suggested Analysis:
#
## 1. Correlation Analysis:
## Calculate the correlation matrix to understand relationships between variables.
#correlation_matrix = merged_df.corr()
#print("Correlation Matrix:")
#print(correlation_matrix)
#
## 2. Heatmap Visualization:
## Visualize the correlation matrix using a heatmap.
#import matplotlib.pyplot as plt
#
#plt.figure(figsize=(12, 8))
#sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
#plt.title("Correlation Heatmap")
#plt.show()
#
## 3. Regional Analysis:
## Group data by regions and calculate summary statistics.
#regional_summary = merged_df.groupby("Regio's").mean()
#print("Regional Summary:")
#print(regional_summary)
#
## 4. Scatter Plot Analysis:
## Visualize relationships between key variables, e.g., GDP per capita vs. online crime rates.
#plt.figure(figsize=(8, 6))
#sns.scatterplot(
#    data=merged_df,
#    x="Bbp per inwoner (euro)",
#    y="Online criminaliteit Totaal slachtoffers (%)",
#    hue="Bevolkingsdichtheid (aantal inwoners per km²)"
#)
#plt.title("GDP per Capita vs. Online Crime Rates")
#plt.xlabel("GDP per Capita (Euro)")
#plt.ylabel("Online Crime Rates (%)")
#plt.legend(title="Population Density")
#plt.show()
#
## 5. Digital Skills Impact:
## Analyze the impact of digital skills on online crime rates.
#plt.figure(figsize=(8, 6))
#sns.boxplot(
#    data=merged_df,
#    x="Digitale vaardigheden",
#    y="Online criminaliteit Totaal slachtoffers (%)"
#)
#plt.title("Impact of Digital Skills on Online Crime Rates")
#plt.xlabel("Digital Skills")
#plt.ylabel("Online Crime Rates (%)")
#plt.show()
#
## 6. Crime Type Analysis:
## Add analysis for other crime types if available in the dataset.
#if "Phishing Totaal slachtoffers (%)" in merged_df.columns:
#    plt.figure(figsize=(8, 6))
#    sns.scatterplot(
#        data=merged_df,
#        x="Bbp per inwoner (euro)",
#        y="Phishing Totaal slachtoffers (%)",
#        hue="Bevolkingsdichtheid (aantal inwoners per km²)"
#    )
#    plt.title("GDP per Capita vs. Phishing Crime Rates")
#    plt.xlabel("GDP per Capita (Euro)")
#    plt.ylabel("Phishing Crime Rates (%)")
#    plt.legend(title="Population Density")
#    plt.show()